36MFCCGAN-01.09.17 BELLOW   - updated on Rah-GPU01.11.10----last ver.01.12.01

In [ ]:
aBOVE FROM 1200 Epochs-36MFCCGAN-LSGAN Without any Perceptual Optimization.01.09.17

In [ ]:
BELOW 1200 Epochs-36MFCCGAN-LSGAN With mYSTOI Perceptual Optimization.01.11.02

In [5]:
data_path='/media/mb/76c865bc-69a9-4151-b7c1-c8463b4fc43b/mb/Datastes/LJSpeech-1.1/wavs'

In [24]:
!pwd

/media


In [16]:
cd home

[Errno 2] No such file or directory: 'home'
/home/mb


In [6]:
cd 

/home/mb


In [28]:
cd media

/media


cd ../

In [39]:
cd ../../../../

/


In [40]:
!pwd

/


In [42]:
cd 'media/mb/76c865bc-69a9-4151-b7c1-c8463b4fc43b/mb/Datastes/LJSpeech-1.1/wavs/'

[Errno 2] No such file or directory: 'media/mb/76c865bc-69a9-4151-b7c1-c8463b4fc43b/mb/Datastes/LJSpeech-1.1/wavs/'
/


In [28]:
cd ../../

/media/behdad/76c865bc-69a9-4151-b7c1-c8463b4fc43b/mb


In [ ]:


#MB:last version 01.09.05 fromr 0 epoch- 36MFCCGAN LSGAN WITHout MCD_Score PERCERP. oPT FOR 1200 ePOCH Without Resume

import librosa
import math
import torch
import torch.utils.data
import torch.nn.functional as F
from torch.nn.utils import weight_norm
from librosa.core import load
#from librosa.core import loadmel_basis
from librosa.util import normalize

from pathlib import Path
import numpy as np
import random
##
import torch.nn as nn
import torch.nn.functional as F
import torch
from librosa.filters import mel as librosa_mel_fn
from torch.nn.utils import weight_norm
import numpy as np

#MB:01.07.26:
from pystoi import stoi
#MB:01.11.10 first day on Rah-GPU-PC
#from pypesq import pesq
#import pesq

#############################
#data_path="wavs" 
#data_path="/home/sbuspeech/SBU_Codes/Datasets/LJSpeech-1.1/wavs/"
#MB:01.11.10 first Rah-Gpu
#data_path="/home/mb/MB_Speech01.11.09/Datastes/LJSpeech-1.1/wavs/"
#data_path="/home/sbuspeech/SBU_Codes/Datasets/LJSpeech-1.1/wavs100 (original)/"
#data_path='/media/mb/76c865bc-69a9-4151-b7c1-c8463b4fc43b/mb/Datastes/LJSpeech-1.1/wavs'
#data_path='/media/mb/76c865bc-69a9-4151-b7c1-c8463b4fc43b/mb/Datastes/LJSpeech-1.1/wavs/'

#MB:01.11.26:
#data_path='/Datasets/LJSpeech-1.1/wavs'
data_path='/home/behdad/MB_Speech01.11.09/Datasets/LJSpeech-1.1/wavs/'


#   /media/mb/76c865bc-69a9-4151-b7c1-c8463b4fc43b/mb/Datastes/LJSpeech-1.1/
def files_to_list(filename):
    """
    Takes a text file of filenames and makes a list of filenames
    """
    with open(filename, encoding="utf-8") as f:
        files = f.readlines()

    files = [f.rstrip() for f in files]
    return files
########
filename=Path(data_path) / "train_files.txt"
files=files_to_list(filename)
#print(type(files))
#print(len(files))
#print(files)
#print('###')
audio_files = [filename.parent / x for x in files]
#print(audio_files )
random.seed(1234)
random.shuffle(audio_files)
##############################################################################
class AudioDataset(torch.utils.data.Dataset):
    """
    This is the main class that calculates the spectrogram and returns the
    spectrogram, audio pair.
    """

    def __init__(self, training_files, segment_length, sampling_rate, augment=True):
        self.sampling_rate = sampling_rate
        self.segment_length = segment_length
        self.audio_files = files_to_list(training_files)
        self.audio_files = [Path(training_files).parent / x for x in self.audio_files]
        random.seed(1234)
        random.shuffle(self.audio_files)
        self.augment = augment

    def __getitem__(self, index):
        # Read audio
        filename = self.audio_files[index]
        audio, sampling_rate = self.load_wav_to_torch(filename)
        # Take segment
        if audio.size(0) >= self.segment_length:
            max_audio_start = audio.size(0) - self.segment_length
            audio_start = random.randint(0, max_audio_start)
            audio = audio[audio_start : audio_start + self.segment_length]
        else:
            audio = F.pad(
                audio, (0, self.segment_length - audio.size(0)), "constant"
            ).data

        # audio = audio / 32768.0
        return audio.unsqueeze(0)

    def __len__(self):
        return len(self.audio_files)

    def load_wav_to_torch(self, full_path):
        """
        Loads wavdata into torch array
        """
        data, sampling_rate = load(full_path, sr=self.sampling_rate)
        data = 0.95 * normalize(data)

        if self.augment:
            amplitude = np.random.uniform(low=0.3, high=1.0)
            data = data * amplitude

        return torch.from_numpy(data).float(), sampling_rate

##############
#######################################################################################################3
import torch.nn as nn
class Audio2Mel(nn.Module):
    def __init__(
        self,
        n_fft=1024,
        hop_length=256,
        win_length=1024,
        sampling_rate=22050,
    
        #n_mel_channels=80,################????????????????????01.05.08 MB?????????????????????????????????????????
        #MB:01.07.25:
        n_mel_channels=36,
        #MB:01.06
        #n_mel_channels=24,
        mel_fmin=0.0,
        mel_fmax=None,
    ):
        super().__init__()
        ##############################################
        # FFT Parameters                              #
        ##############################################
        window = torch.hann_window(win_length).float()
        mel_basis = librosa_mel_fn(
            sampling_rate, n_fft, n_mel_channels, mel_fmin, mel_fmax
        )
        mel_basis = torch.from_numpy(mel_basis).float()
        self.register_buffer("mel_basis", mel_basis)
        self.register_buffer("window", window)
        self.n_fft = n_fft
        self.hop_length = hop_length
        self.win_length = win_length
        self.sampling_rate = sampling_rate
        self.n_mel_channels = n_mel_channels

    def forward(self, audio):
        p = (self.n_fft - self.hop_length) // 2
        audio = F.pad(audio, (p, p), "reflect").squeeze(1)
        fft = torch.stft(
            audio,
            n_fft=self.n_fft,
            hop_length=self.hop_length,
            win_length=self.win_length,
            window=self.window,
            center=False,
        )
        real_part, imag_part = fft.unbind(-1)
        magnitude = torch.sqrt(real_part ** 2 + imag_part ** 2)
        mel_output = torch.matmul(self.mel_basis, magnitude)
        log_mel_spec = torch.log10(torch.clamp(mel_output, min=1e-5))
        return log_mel_spec
######
###################################################################################
from torch.nn.utils import weight_norm

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find("BatchNorm2d") != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)


def WNConv1d(*args, **kwargs):
    return weight_norm(nn.Conv1d(*args, **kwargs))


def WNConvTranspose1d(*args, **kwargs):
    return weight_norm(nn.ConvTranspose1d(*args, **kwargs))
################
class ResnetBlock(nn.Module):
    def __init__(self, dim, dilation=1):
        super().__init__()
        self.block = nn.Sequential(
            nn.LeakyReLU(0.2),
            nn.ReflectionPad1d(dilation),
            WNConv1d(dim, dim, kernel_size=3, dilation=dilation),
            nn.LeakyReLU(0.2),
            WNConv1d(dim, dim, kernel_size=1),
        )
        self.shortcut = WNConv1d(dim, dim, kernel_size=1)

    def forward(self, x):
        return self.shortcut(x) + self.block(x)
#######
####################################################################################################
class Generator(nn.Module):
    def __init__(self, input_size, ngf, n_residual_layers):
        super().__init__()
        ratios = [8, 8, 2, 2]
        self.hop_length = np.prod(ratios)
        mult = int(2 ** len(ratios))

        model = [
            nn.ReflectionPad1d(3),
            WNConv1d(input_size, mult * ngf, kernel_size=7, padding=0),
        ]
        MRH_in=32
        # Upsample to raw audio scale
        for i, r in enumerate(ratios):
            #print('mult= ',mult)
            model += [
                nn.LeakyReLU(0.2),
                WNConvTranspose1d(
                    mult * ngf,
                    mult * ngf // 2,
                    kernel_size=r * 2,
                    stride=r,
                    padding=r // 2 + r % 2,
                    output_padding=r % 2,
                ),
            ]
            #MRH
            #print('output channel= ',mult * ngf // 2)
            kernel_size=r * 2
            stride=r
            padding=r // 2 + r % 2
            output_padding=r % 2
            MRH_dilation=1
            MRH_length=(MRH_in-1)*stride-2*padding +MRH_dilation*(kernel_size-1)+ output_padding + 1
            MRH_in=MRH_length
            #print(input_size)
            #print("MRH calculated output Length= ",MRH_length)

            for j in range(n_residual_layers):
                model += [ResnetBlock(mult * ngf // 2, dilation=3 ** j)]

            mult //= 2

        model += [
            nn.LeakyReLU(0.2),
            nn.ReflectionPad1d(3),
            WNConv1d(ngf, 1, kernel_size=7, padding=0),
            nn.Tanh(),
        ]

        self.model = nn.Sequential(*model)
        self.apply(weights_init)

    def forward(self, x):
        #print(x.size())
        return self.model(x)
#######
############################################################################
class MyModule(nn.Module):
    def __init__(self):
        super(MyModule, self).__init__()
        self.choices = nn.ModuleDict({
                'conv': nn.Conv2d(10, 10, 3),
                'pool': nn.MaxPool2d(3)
        })
        self.activations = nn.ModuleDict([
                ['lrelu', nn.LeakyReLU()],
                ['prelu', nn.PReLU()]
        ])

    def forward(self, x, choice='conv', act='prelu'):
        x = self.choices[choice](x)
        #print(x.size())
        x = self.activations[act](x)
        #print(x.size())
        return x
#####################################################################################################
class NLayerDiscriminator(nn.Module):
    def __init__(self, ndf, n_layers, downsampling_factor):
        super().__init__()
        model = nn.ModuleDict()

        model["layer_0"] = nn.Sequential(
            nn.ReflectionPad1d(7),
            WNConv1d(1, ndf, kernel_size=15),
            nn.LeakyReLU(0.2, True),
        )

        nf = ndf
        stride = downsampling_factor
        for n in range(1, n_layers + 1):
            nf_prev = nf
            nf = min(nf * stride, 1024)

            model["layer_%d" % n] = nn.Sequential(
                WNConv1d(
                    nf_prev,
                    nf,
                    kernel_size=stride * 10 + 1,
                    stride=stride,
                    padding=stride * 5,
                    groups=nf_prev // 4,
                ),
                nn.LeakyReLU(0.2, True),
            )

        nf = min(nf * 2, 1024)
        model["layer_%d" % (n_layers + 1)] = nn.Sequential(
            WNConv1d(nf_prev, nf, kernel_size=5, stride=1, padding=2),
            nn.LeakyReLU(0.2, True),
        )

        model["layer_%d" % (n_layers + 2)] = WNConv1d(
            nf, 1, kernel_size=3, stride=1, padding=1
        )

        self.model = model

    def forward(self, x):
        results = []
        for key, layer in self.model.items():
            x = layer(x)
            results.append(x)
        return results


class Discriminator(nn.Module):
    def __init__(self, num_D, ndf, n_layers, downsampling_factor):
        super().__init__()
        self.model = nn.ModuleDict()
        for i in range(num_D):
            self.model[f"disc_{i}"] = NLayerDiscriminator(
                ndf, n_layers, downsampling_factor
            )

        self.downsample = nn.AvgPool1d(4, stride=2, padding=1, count_include_pad=False)
        self.apply(weights_init)

    def forward(self, x):
        results = []
        for key, disc in self.model.items():
            #print('key= ', key)
            #print('discrimiator= ',disc)
            results.append(disc(x))
            #print('Length results== ',len(results))
            #print('results', results)
            x = self.downsample(x)
        return results
#######################################################################
# from mel2wav.utils import save_sample

import scipy.io.wavfile
def save_sample(file_path, sampling_rate, audio):
    """Helper function to save sample

    Args:
        file_path (str or pathlib.Path): save file path
        sampling_rate (int): sampling rate of audio (usually 22050)
        audio (torch.FloatTensor): torch array containing audio in [-1, 1]
    """
    audio = (audio.numpy() * 32768).astype("int16")
    scipy.io.wavfile.write(file_path, sampling_rate, audio)
#########################################################################

class Audio2MFCC(nn.Module):
    def __init__(
        self,
        n_fft=1024,
        hop_length=256,
        win_length=1024,
        sampling_rate=22050,
        
        #MB: 01.04.01 %1 %%%%%%%%%%%%%%%%%%%%%----------->>>>>>
        n_mel_channels=36,
        mel_fmin=0.0,
        mel_fmax=None,
    ):
        super().__init__()
        ##############################################
        # FFT Parameters                              #
        ##############################################
        #window = torch.hann_window(win_length).float()
        #mel_basis = librosa_mel_fn(
        #    sampling_rate, n_fft, n_mel_channels, mel_fmin, mel_fmax
        #)
        #mel_basis = torch.from_numpy(mel_basis).float()
        #self.register_buffer("mel_basis", mel_basis)
        #self.register_buffer("window", window)
        self.n_fft = n_fft
        self.hop_length = hop_length
        self.win_length = win_length
        self.sampling_rate = sampling_rate
        self.n_mel_channels = n_mel_channels

    def forward(self, x):
        y=x[0,:,:]
        #    print('x= ',x.size())
        #    print('y= ',y.size())
        z=y.cpu().numpy()
        #print(type(z))
        #print('z shape === ',z.shape)
        w=z.reshape(x.size(2),)
        #print('w shape= ',w.shape)
        
        #MB: 01.04.02 %2 %%%%%%%%%%%%%%%%%%%%%-------------------->>>>>>>
        mfccs = librosa.feature.mfcc(w, sr=self.sampling_rate,n_mfcc=36, n_fft=1024, hop_length=258, win_length=1024)
        
        #print('mfccs shape== ',mfccs.shape)
        frame_number=mfccs.shape[1]
        #print(self.n_mel_channels,frame_number)
        #print('x.size(0)= ',type(x.size(0)))
        t = torch.randn(x.size(0),self.n_mel_channels,frame_number)
        #print('t.size()== ',t.size())
        #print(x)
        ####
        
        ####
        for i in np.arange(x.size(0)):
            #print(i)
            y=x[i,:,:]
            #print('y= ',y.size())
            z=y.cpu().numpy()
            #print(type(z))
            #print('z shape === ',z.shape)
            w=z.reshape(x.size(2),)
            #print('w shape=== ',w.shape)
            #MB: 01.04.02 %3 %%%%n_mfcc =24%%%%%%%%%%%%%%%%%------------>>>>>>
            mfccs = librosa.feature.mfcc(w, sr=self.sampling_rate,n_mfcc=36, n_fft=1024, hop_length=258, win_length=1024)
           
            ################################## MB:01.05.31 added this block ###########################################
            #print("mfccs.shape ",mfccs.shape )
            #              Audio=librosa.feature.inverse.mfcc_to_audio(mfc
            #print("Audio file resynthesized :Audio_%d.wav" %i )cs, n_mels=256, dct_type=2, norm='ortho', ref=1.0, lifter=0)
            #Audio=librosa.feature.inverse.mfcc_to_audio(mfccs, n_mels=128, dct_type=2, norm='ortho', ref=1.0, lifter=0)
            #MB:01.03.22:
            #Audio = (Audio.numpy() * 32768).astype("int16")
            #       I=str(i)
            #       path_file="/home/sbuspeech/SBU_Codes/MBehdad/CycleGAN-VC2/MRHlogs13/Audio_"+I+".wav"
            # scipy.io.wavfile.write(path_file , 22050 ,Audio)
            #original_%d.wav" % i
            #write(filename=os.path.join(output_A_dir, os.path.basename(file)), rate=sampling_rate, data=wav_transformed)
            ########## 3)-------------->
            #MB:01.03.20 :
            ############################################################################################################   
            #print(mfccs.shape[1])
            #print("\n mfccs.shape[1] :",mfccs.shape[1])    
            #print(mfccs.shape[1])
            Q=torch.tensor(mfccs)
            #print('Q== ',Q.size())
            #print(t[i,:,:])
            t[i,:,:]=Q
            
            #print(t[i,:,:])
        return t
######

##################################
############ 4 train.py ###########
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

import yaml
import numpy as np
import time
import argparse
from pathlib import Path

#################
# MRH:
#MB:01.11.13 New Saving area on GPU1080
#save_path='MRHlogs13/MRH-Baseline'
#save_path='/media/mb/76c865bc-69a9-4151-b7c1-c8463b4fc43b/mb/Datasets/MFCCGAN/MRHlogs13/MRH-Baseline'
#MB:01.11.26:
#save_path='/home/behdad/MB_Speech01.11.09/Datasets/MFCCGAN/MRHlogs13/MRH-Baseline/'
#MB:01.12.01
save_path='/media/behdad/76c865bc-69a9-4151-b7c1-c8463b4fc43b/mb/Datasets/MFCCGAN/MRHlogs13/MRH-Baseline/'
load_path=None   # >>>>>> 14000422 >>>> /home/sbuspeech/SBU_Codes/MBehdad/CycleGAN-VC2/MRHlogs13
#load_path='/home/sbuspeech/SBU_Codes/MBehdad/CycleGAN-VC2/MRHlogs13//MRH-Baseline/'
#MB:01.12.01:
#load_path='/media/behdad/76c865bc-69a9-4151-b7c1-c8463b4fc43b/mb/Datasets/MFCCGAN/MRHlogs13//MRH-Baseline/'

#print("Training is Resumed")
#/home/sbuspeech/SBU_Codes/MBehdad/CycleGAN-VC2/MRHlogs13/MRH-Baseline

########3n_mel_channels=80
#MB: 01.04.02 %4 %%%%%%%%%%%%%%%%%%%%%------->>>>>>>>>>
n_mel_channels=36

ngf=32
n_residual_layers=3 
ndf=16 
num_D=3
n_layers_D=4 
downsamp_factor=4 
lambda_feat=10
#cond_disc", action="store_true")
#MB:01.11.10 RahGPUSys first day
#data_path="/home/sbuspeech/SBU_Codes/Datasets/LJSpeech-1.1/wavs/"
#data_path="/media/mb/76c865bc-69a9-4151-b7c1-c8463b4fc43b/mb/Datasets/LJSpeech-1.1/wavs/"

#data_path='/media/behdad/76c865bc-69a9-4151-b7c1-c8463b4fc43b/mb/Datasets/LJSpeech-1.1/wavs'
#data_path='/Datasets/LJSpeech-1.1/wavs/'
data_path='/home/behdad/MB_Speech01.11.09/Datasets/LJSpeech-1.1/wavs'

batch_size=16 
seq_len=8192
#MB:01.04.07 -1200epochs
#epochs=30000
epochs=3500
log_interval=100
#MB:01.06.18 1000 to 81300
#save_interval=1000 
#save_interval=81300
save_interval=40650
n_test_samples=100
#####################
MRH_Dict={}
MRH_Dict=dict({
    'save_path':'MRHlogs13/MRH-Baseline',
    'load_path':None, 
    'n_mel_channels':36,
    'ngf':32,
    'n_residual_layers':3,
    'ndf':16,
    'num_D':3, 
    'n_layers_D':4,
    'downsamp_factor':4,
    'lambda_feat':10,
    #cond_disc", action="store_true")

    'data_path':'wavs/',
    'batch_size':16,
    'seq_len':8192,
    'epochs':1200,
    'log_interval':100,
    'save_interval':81300,
    'n_test_samples':1
})

"""
************************************************* Main *************************************************
"""
import decimal
def main():
    
    root = Path(save_path)
    load_root = Path(load_path) if load_path else None # >>>>> 14000422
    root.mkdir(parents=True, exist_ok=True)

    ####################################
    # Dump arguments and create logger #
    ####################################
    with open(root / "args.yml", "w") as f:
        yaml.dump(MRH_Dict, f)
    writer = SummaryWriter(str(root))

    #######################
    # Load PyTorch Models #
    #######################
    netG = Generator(n_mel_channels, ngf, n_residual_layers).cuda()
    netD = Discriminator(num_D, ndf, n_layers_D, downsamp_factor).cuda()
    #fft = Audio2Mel(n_mel_channels=n_mel_channels).cuda()
    hop_length=257
    myMFCC = Audio2MFCC(n_mel_channels=n_mel_channels,hop_length=hop_length).cuda()
    #print(netG)
    #print(netD)
    

    #####################
    # Create optimizers #
    #####################
    optG = torch.optim.Adam(netG.parameters(), lr=1e-4, betas=(0.5, 0.9))
    optD = torch.optim.Adam(netD.parameters(), lr=1e-4, betas=(0.5, 0.9))

    if load_root and load_root.exists():
        netG.load_state_dict(torch.load(load_root / "netG.pt"))
        optG.load_state_dict(torch.load(load_root / "optG.pt"))
        netD.load_state_dict(torch.load(load_root / "netD.pt"))
        optD.load_state_dict(torch.load(load_root / "optD.pt"))
        #MB:01.05.05 print added
        #print("\nMB:***Resume after 653 epoch *** ")
        
    #######################
    # Create data loaders #
    #######################
    train_set = AudioDataset(
        Path(data_path) / "train_files.txt", seq_len, sampling_rate=22050
    )
    test_set = AudioDataset(
        Path(data_path) / "test_files.txt",
        22050 * 4,
        sampling_rate=22050,
        augment=False,
    )

    train_loader = DataLoader(train_set, batch_size=batch_size, num_workers=4)
    test_loader = DataLoader(test_set, batch_size=1)
    
    print('##################')

    ##########################
    # Dumping original audio #
    ##########################
    test_voc = []
    test_audio = []
    for i, x_t in enumerate(test_loader):
        #print('i== ',i)
        #x_t = x_t.cuda()
        #print('x_t== ',x_t)
        #print('x_t.size()== ',x_t.size())
        #s_t = fft(x_t).detach()
        s_t=myMFCC(x_t).detach()
        s_t=s_t.cuda()
        #print('s_t===',s_t)
        #s_t2=myMFCC(x_t).detach()
        #print('s_t2===',s_t2)
        test_voc.append(s_t.cuda()) #test_voc.append(s_t.cuda())
        test_audio.append(x_t)

        audio = x_t.squeeze().cpu()
        save_sample(root / ("original_%d.wav" % i), 22050, audio)
        writer.add_audio("original/sample_%d.wav" % i, audio, 0, sample_rate=22050)
        #
        #print("Producing an original wav file:" , root/ ("original_%d.wav" % i))

        if i == n_test_samples - 1:
            break

    costs = []
    start = time.time()

    # enable cudnn autotuner to speed up training
    torch.backends.cudnn.benchmark = True

    best_mel_reconst = 1000000
    steps = 0
    #MB:01.07.24 Added:
    #mystoi=0.1
    
    print("\nTraining 36MFCCGAN by LSGAN with stoi Percep.Opt: 01.11.02")
    print('%%%%%%%%%%%%%%%%%%%%%%%')
    for epoch in range(1, epochs + 1):
        #print(epochs+1)
        print('epoch== ',epoch)
        ####
        #if epoch%50==0:
            #mystoi+=0.1
            ##print("mystoi: ",mystoi)
        ###
        #MB:01.07.25
        mystoi=0
        mystoi_mean=0
        mcd_score=0
        mcd_mean=0
        
        for iterno, x_t in enumerate(train_loader):
            #MB: 01.04.02 cancel iterno print
            #print('iterno= ',iterno)
            #x_t = x_t.cuda()
            #s_t = fft(x_t).detach()
            #print('x_t.size()= ',x_t.size())
            
            s_t=myMFCC(x_t).detach()
            
            #MB:01.07.24
            #   print('s_t size= ',s_t.size())
            s_t=s_t.cuda()
            #x_t = x_t.cuda()
            #print('type(s_t)= ',type(s_t))
            #print('s_t.size()= ',s_t.size())
            x_pred_t = netG(s_t.cuda()) #x_pred_t = netG(s_t.cuda())
            #print('ggg')

            with torch.no_grad():
                #print('x_pred_t.size()= ',x_pred_t.size())
                
                s_pred_t = myMFCC(x_pred_t.detach())
                #s_pred_t = myMFCC(x_pred_t.detach())
                s_pred_t =s_pred_t.cuda()
                #print('s_t.size()=== ',s_t.size())
                #print('s_pred_t.size()==== ',s_pred_t.size())
                s_error = F.l1_loss(s_t, s_pred_t).item()

            #print('# Train Discriminator #')
            #######################
            # Train Discriminator #
            #######################
            ###
                                   
            #MB:01.07.21 MCD Assessment of Real and Fake For Discriminator MCD_OPT
            scaler = 10.0 / np.log(10.0) * np.sqrt(2)
            distortion=(s_t -  s_pred_t)[:, 1:, :]   
            mcd=distortion.pow(2.0).sum(dim=-1).sqrt().mean(dim=-1)*scaler    
            #print("\nmcd.size():",mcd.size())
            #print("mcd:",mcd)
            #x1=mcd.clone().detach()
            x1=mcd.detach().cpu()
            #  print(x1.device)
            #mcd_scaler= x1.item()
            mcd_mean= x1.mean()
            mcd_max=1000
            mcd_score=abs(mcd_max- mcd_mean)/ mcd_max
            """
            print("mcd_mean:",mcd_mean )
            print("mcd_score:",mcd_score)
            #print("\n")
            """
            #mcd_score=math.exp(-1*mcd_RFA_scaler)
            """
            #MB:01.07.23:For STOI
            """
            orig=x_t.detach()
            pred=x_pred_t.detach().cpu()
            #pred=x_pred_t[0,0,:].cpu
            #batch_num=orig[:,0,0].shape()
            batch_num=orig.shape[0]
            #print("orig.shape",orig.shape[0])
            #print("pred.shape",pred.shape[0])
            #print("batch_num",batch_num)
            mystoi=0
            mystoi_mean=0
            
            #MB:01.07.25:
            orig_sig=torch.flatten(x_t)
            pred_sig=torch.flatten(pred)
            """
            print("\norig_sig.shape",orig_sig.shape   )
            print("\norig_sig.size()",orig_sig.size() )
            print("\npred_sig.shape",pred_sig.shape   )
            print("\npred_sig.size()",pred_sig.size() )
            
            save_sample(root / ("original_forPlay_%d.wav" % i), 22050, orig_sig)
            save_sample(root / ("predicted_forPlay_%d.wav" % i), 22050, pred_sig)
            #NISQA Assessment:01.07.25
            #output = sp.getoutput(' python run_predict.py --mode predict_file --pretrained_model weights/nisqa_tts.tar --deg /home/sbuspeech/SBU_Codes/MBehdad/CycleGAN-VC2/MRHlogs13/MRH-Baseline/predicted_forPlay_0.wav --output_dir MB-results ')
            #print("\nnisqa-tts : generated file naturalness:", output )
            #print("\nnisqa-tts : converted file naturalness:\n", output[162:170] )
            #nisqatts_score_str=output[162:170]
            #nisqatts_score[i] = float( nisqatts_score_str)
            #print("\nnisqa-tts:converted  nisqatts_score[%d]  :" %i,  nisqatts_score[i] )
            
            """
            import decimal
            mystoi = stoi( orig_sig, pred_sig , 22050, extended=False)  #*******************************************stoi of flattened
            #mystoi=mystoi.quantize(decimal.Decimal(‘0.000’))
            #print("\nmystoi of 16 parts, flattened:",mystoi)
            
            for i in range(batch_num):
                orig1=orig[i,0,:]
                pred1=pred[i,0,:]
                
                #save_sample(root / ("original_forSTOI_%d.wav" % i), 22050, orig1)
                
                #print(orig1.shape)
                #print(pred1.shape)
                #print(orig1.device)
                #print(pred1.device)
                #mystoi = stoi( orig1 ,pred1 , 22050, extended=False)
                """
                mystoi = stoi( pred1 ,orig1 , 22050, extended=False)
                print("batch mystoi:",mystoi)
                mystoi+=mystoi
                #print("mystoi: " ,mystoi)
                #MB:01.07.24:
                save_sample(root / ("generated_forSTOI_%d.wav" % i), 22050, pred1)
                """
                #save_sample(root / ("***generated_forSTOI_%d.wav" % i), 22050, pred_audio1)
            #mystoi_mean=mystoi/batch_num
            #print("mystoi_mean: ",mystoi_mean)    
            
            #if epoch == 0 and num_iterations ==0: print("\n%% mcd_scaler:%%",mcd_scaler,"mcd_score:" ,mcd_score)
            """
            #MB:01.07.24
            #distortion_RFA=(real_A -  fake_A)[:, 1:, :]   
            #mcd_RFA=distortion_RFA.pow(2.0).sum(dim=-1).sqrt().mean(dim=-1)*scaler    
            #x1=mcd_RFA.clone().detach().requires_grad_(True)
            #mcd_RFA_scaler= x1.item()
            #mcd_max=83
            #mcd_score_RFA=math.exp(-1*mcd_RFA_scaler)
            """
            
                          
            ###
            D_fake_det = netD(x_pred_t.cuda().detach()) #netD(x_pred_t.cuda
            #print(len(D_fake_det))
            #print(D_fake_det)
            D_real = netD(x_t.cuda()) # netD(x_t)

            loss_D = 0
            for scale in D_fake_det:
                #print(scale)
                #print(len(scale))
                #print('$$$$$$$$$$')
                #print('scale[-1]= ',scale[-1])
                #mrh1=F.relu(1 + scale[-1])
                #print('mrh1= ',mrh1)
                #mrh2=F.relu(1 + scale[-1]).mean()
                #print('mrh2= ',mrh2)
                """
                #MB: 01.07.21:
                print("********************************************")
                print("\nx_pred_t",x_pred_t.size() )
                print("\nx_pred_t",x_pred_t.shape)
                print("\nx_pred_t",x_pred_t.shape[0])
                print("\nx_pred_t",x_pred_t.shape[1])
                print("\nx_pred_t",x_pred_t.shape[2])
            
                print("x_t.size()",x_t.size())
                print("\nx_t.shape",x_t.shape)
                """                        
                #loss_D += F.relu(1 + scale[-1]).mean()
                #loss_D += F.relu( mcd_score + scale[-1]).mean()
                #loss_D += F.relu(mystoi + scale[-1]).mean()
                #print('##')
                #print(loss_D)
                #MB:01.07.28:changed following:
                #loss_D += ((0 - scale[-1])**2).mean()  #Least Squre
                loss_D += ((mystoi - scale[-1])**2).mean()#*******************************
                #MB:01.08.01:
                #loss_D += ((mcd_score - scale[-1])**2).mean()#*******************************
                

            for scale in D_real:
                #loss_D += F.relu(1 - scale[-1]).mean()
                #MB:01.07.28:changed following:
                loss_D +=((1 - scale[-1])**2).mean() #***************************************

            #MB:01.07.27
            #loss_D=0.5*loss_D+0.5(1-mystoi)
            netD.zero_grad()
            loss_D.backward()
            optD.step()

            #print('# Train Generator #')
            ###################
            # Train Generator #
            ###################
            D_fake = netD(x_pred_t.cuda())
        
            loss_G = 0
            for scale in D_fake:
                #loss_G += -scale[-1].mean()
                #MB:01.07.28 Changed following:
                loss_G += ((1-scale[-1])**2).mean()  #*************************************
                

            loss_feat = 0
            feat_weights = 4.0 / (n_layers_D + 1)
            D_weights = 1.0 / num_D
            wt = D_weights * feat_weights
            for i in range(num_D):
                for j in range(len(D_fake[i]) - 1):
                    loss_feat += wt * F.l1_loss(D_fake[i][j], D_real[i][j].detach())

            netG.zero_grad()
            (loss_G + lambda_feat * loss_feat).backward()
            optG.step()

            ######################
            # Update tensorboard #
            ######################
            costs.append([loss_D.item(), loss_G.item(), loss_feat.item(), s_error])

            writer.add_scalar("loss/discriminator", costs[-1][0], steps)
            writer.add_scalar("loss/generator", costs[-1][1], steps)
            writer.add_scalar("loss/feature_matching", costs[-1][2], steps)
            writer.add_scalar("loss/mel_reconstruction", costs[-1][3], steps)
            steps += 1
            #print(steps)
            

            if steps % save_interval == 0:
                st = time.time()
                with torch.no_grad():
                    for i, (voc, _) in enumerate(zip(test_voc, test_audio)):
                        #print('test_voc.size() ==== >',test_voc.size())
                        #print('test_audio.size() ==== >',test_audio.size())
                        pred_audio = netG(voc)
                        pred_audio = pred_audio.squeeze().cpu()
                        save_sample(root / ("generated_%d.wav" % i), 22050, pred_audio)
                        writer.add_audio(
                            "generated/sample_%d.wav" % i,
                            pred_audio,
                            epoch,
                            sample_rate=22050,
                        )

                torch.save(netG.state_dict(), root / "netG.pt")
                torch.save(optG.state_dict(), root / "optG.pt")

                torch.save(netD.state_dict(), root / "netD.pt")
                torch.save(optD.state_dict(), root / "optD.pt")

                if np.asarray(costs).mean(0)[-1] < best_mel_reconst:
                    best_mel_reconst = np.asarray(costs).mean(0)[-1]
                    torch.save(netD.state_dict(), root / "best_netD.pt")
                    torch.save(netG.state_dict(), root / "best_netG.pt")

                print("\n **Took %5.4fs to generate samples" % (time.time() - st))
                print("-" * 100)
            mystoi1= round(mystoi, 3)
            #mcd_score1=mcd_score.numpy.cpu()
            #mcd_score2= round(mcd_score1,3)
            #print("Training 36MFCCGAN with  Linear(Max=1000) MCD.Opt")
            #print("Training 36MFCCGAN without mcd.opt")
            if steps % log_interval == 0:
                print( "mystoi:",mystoi1, "mcd_score:",mcd_score,
                #print( "mystoi_mean:",mystoi_mean, "mcd_score:",mcd_score,
                #print(
                    "Epoch {} | Iters {} / {} | ms/batch {:5.2f} | loss {}".format(
                        epoch,
                        iterno,
                        len(train_loader),
                        1000 * (time.time() - start) / log_interval,
                        np.asarray(costs).mean(0),
                    )
                )
                costs = []
                start = time.time()

if __name__ == "__main__":
    import time
    import datetime
    #print('111')
    e = datetime.datetime.now()
    print ("Today's date:  = %s/%s/%s" % (e.day, e.month, e.year))
    print ("The time is now: = %s:%s:%s" % (e.hour, e.minute, e.second))
    main()

outputn

Today's date:  = 20/2/2023
The time is now: = 13:53:6
##################


 cd '/media/behdad/76c865bc-69a9-4151-b7c1-c8463b4fc43b/mb/Datasets/LJSpeech-1.1/wavs'

In [46]:
!pwd

/media/behdad/76c865bc-69a9-4151-b7c1-c8463b4fc43b/mb


In [47]:
cd /../../../../

/


In [48]:
cd home

/home


In [39]:
cd mb

/media/behdad/76c865bc-69a9-4151-b7c1-c8463b4fc43b/mb


In [40]:
cd Datasets

[Errno 2] No such file or directory: 'Datasets'
/media/behdad/76c865bc-69a9-4151-b7c1-c8463b4fc43b/mb


above: 01.11.13 Thursday- Yesterday the First 36MFCCGAN IN GPU1080 - 36MFCCGAN With Stoi.Opt for 1500 epoch

In [4]:
!pwd

/home/behdad/MB_Speech01.11.09/MFCCGAN


In [12]:
cd ../

/


In [7]:
import numba
print(numba.__version__)

0.55.1


In [6]:
import numpy
numpy.__version__


'1.21.5'

In [ ]:
data_path="/media/mb/76c865bc-69a9-4151-b7c1-c8463b4fc43b/mb/Datasets/LJSpeech-1.1/wavs/"

In [8]:
import torch
print(torch.__version__)

1.13.1


In [16]:
import tensorboard

In [2]:
import librosa
print(librosa.__version__)

ImportError: Numba needs NumPy 1.21 or less

In [2]:
data_path="/media/mb/76c865bc-69a9-4151-b7c1-c8463b4fc43b/mb/Datasets/LJSpeech-1.1/wavs/"

In [3]:
cd "/media/mb/76c865bc-69a9-4151-b7c1-c8463b4fc43b/mb/Datasets/LJSpeech-1.1/wavs/"

/media/mb/76c865bc-69a9-4151-b7c1-c8463b4fc43b/mb/Datastes/LJSpeech-1.1/wavs


In [4]:
ls

LJ001-0001.wav  LJ010-0031.wav  LJ018-0106.wav  LJ029-0087.wav  LJ040-0051.wav
LJ001-0002.wav  LJ010-0032.wav  LJ018-0107.wav  LJ029-0088.wav  LJ040-0052.wav
LJ001-0003.wav  LJ010-0033.wav  LJ018-0108.wav  LJ029-0089.wav  LJ040-0053.wav
LJ001-0004.wav  LJ010-0034.wav  LJ018-0109.wav  LJ029-0090.wav  LJ040-0054.wav
LJ001-0005.wav  LJ010-0035.wav  LJ018-0110.wav  LJ029-0091.wav  LJ040-0055.wav
LJ001-0006.wav  LJ010-0036.wav  LJ018-0111.wav  LJ029-0092.wav  LJ040-0056.wav
LJ001-0007.wav  LJ010-0037.wav  LJ018-0112.wav  LJ029-0093.wav  LJ040-0057.wav
LJ001-0008.wav  LJ010-0038.wav  LJ018-0113.wav  LJ029-0094.wav  LJ040-0058.wav
LJ001-0009.wav  LJ010-0039.wav  LJ018-0114.wav  LJ029-0095.wav  LJ040-0059.wav
LJ001-0010.wav  LJ010-0040.wav  LJ018-0115.wav  LJ029-0096.wav  LJ040-0060.wav
LJ001-0011.wav  LJ010-0041.wav  LJ018-0116.wav  LJ029-0097.wav  LJ040-0061.wav
LJ001-0012.wav  LJ010-0042.wav  LJ018-0117.wav  LJ029-0098.wav  LJ040-0062.wav
LJ001-0013.wav  LJ010-0043.wav  LJ018-0118.wav  LJ02

In [9]:
!pip3 uninstall numpy

Found existing installation: numpy 1.21.5
Uninstalling numpy-1.21.5:
  Would remove:
    /home/behdad/anaconda3/bin/f2py
    /home/behdad/anaconda3/bin/f2py3
    /home/behdad/anaconda3/bin/f2py3.9
    /home/behdad/anaconda3/lib/python3.9/site-packages/numpy-1.21.5.dist-info/*
    /home/behdad/anaconda3/lib/python3.9/site-packages/numpy/*
Proceed (Y/n)? ^C
ERROR: Operation cancelled by user


In [5]:
import numba
print(numba.__version__)


0.55.1
